<a href="https://colab.research.google.com/github/byunsy/bioinformatics-algorithms-py/blob/main/BA_4G.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Leaderboard Cyclopeptide Sequencing Problem

### Function

In [ ]:
def Expand(pep):

    amino_acid = ['G','A','S','P','V','T','C','I','L','N','D','K','Q','E','M','H','F','R','Y','W']
    new_pep = []

    for i in pep:
        for amino in amino_acid:
            new_pep.append(i+amino)

    return new_pep

In [ ]:
def Mass(pep, amino_acid_mass):

    mass = 0
    for i in pep:
        mass += amino_acid_mass[i]

    return mass

In [ ]:
def LinearSpectrum(peptide, amino_acid_mass):
    
    prefix_mass = [0]
    for amino in peptide:
        prefix_mass.append(prefix_mass[-1] + amino_acid_mass[amino])

    linear_spectrum = [0]
    for i in range(len(peptide)):
        j = i + 1
        while j <= len(peptide):
            linear_spectrum.append(prefix_mass[j]-prefix_mass[i])
            j += 1
    
    return sorted(linear_spectrum)

In [ ]:
amino_acid_mass = {
    'G' : 57, 'A' : 71, 'S' : 87, 'P' : 97, 'V' : 99, 'T' : 101, 'C' : 103, 'I' : 113, 'L' : 113, 'N' : 114,
    'D' : 115, 'K' : 128, 'Q' : 128, 'E' : 129, 'M' : 131, 'H' : 137, 'F' : 147, 'R' : 156, 'Y' : 163, 'W' : 186 
}

from collections import Counter

def LinearScore(peptide, spectrum):

    pep_spectrum = LinearSpectrum(peptide, amino_acid_mass)

    pep_counter = Counter(pep_spectrum) 
    spc_counter = Counter(spectrum)
    score = 0

    for i in set(pep_spectrum):
        if i in spectrum:
            if pep_counter[i] <= spc_counter[i]:
                score += pep_counter[i]
            else: # pep_counter[i] > spc_counter[i]
                score += spc_counter[i]

    return score

In [ ]:
def CyclicSpectrum(peptide, amino_acid_mass):
    
    prefix_mass = [0]
    for amino in peptide:
        prefix_mass.append(prefix_mass[-1] + amino_acid_mass[amino])

    peptide_mass = prefix_mass[len(peptide)]
    cyclic_spectrum = [0]
    for i in range(len(peptide)):
        j = i + 1
        while j <= len(peptide):
            cyclic_spectrum.append(prefix_mass[j]-prefix_mass[i])
            if i > 0 and j < len(peptide):
                cyclic_spectrum.append(peptide_mass - (prefix_mass[j]-prefix_mass[i]))
            j += 1
    
    return sorted(cyclic_spectrum)

In [ ]:
amino_acid_mass = {
    'G' : 57, 'A' : 71, 'S' : 87, 'P' : 97, 'V' : 99, 'T' : 101, 'C' : 103, 'I' : 113, 'L' : 113, 'N' : 114,
    'D' : 115, 'K' : 128, 'Q' : 128, 'E' : 129, 'M' : 131, 'H' : 137, 'F' : 147, 'R' : 156, 'Y' : 163, 'W' : 186 
}

from collections import Counter

def CyclopeptideScore(peptide, spectrum):

    pep_spectrum = CyclicSpectrum(peptide, amino_acid_mass)

    pep_counter = Counter(pep_spectrum) 
    spc_counter = Counter(spectrum)
    score = 0

    for i in set(pep_spectrum):
        if i in spectrum:
            if pep_counter[i] <= spc_counter[i]:
                score += pep_counter[i]
            else: # pep_counter[i] > spc_counter[i]
                score += spc_counter[i]

    return score

In [ ]:
def DisplayPepMass(pep):

    string = ""
    for amino in pep:
        string += str(Mass(amino, amino_acid_mass)) + "-"

    return string[:-1]

In [53]:
# def Trim(leaderboard, spectrum, n):

#     linear_scores = {}
#     for pep in leaderboard:
#         lin_score = LinearScore(pep, spectrum)
#         linear_scores[pep] = lin_score
    
#     sorted_LB = dict(sorted(linear_scores.items(), key=lambda x:x[1], reverse=True))
    
#     lb_pep = list(sorted_LB.keys())
#     lb_scores = list(sorted_LB.values())

#     cutoff_idx = n

#     for i in range(cutoff_idx, len(lb_scores)):
#         if lb_scores[i] < lb_scores[i-1]:
#             break
#         else:
#             cutoff_idx += 1

#     return lb_pep[:cutoff_idx]

In [61]:
def Trim(leaderboard, spectrum, n):

    linear_scores = []
    for pep in leaderboard:
        linear_scores.append(LinearScore(pep, spectrum))

    lb_pep    = [p for _,p in sorted(zip(linear_scores,leaderboard), reverse=True)]
    lb_scores = [s for s,_ in sorted(zip(linear_scores,leaderboard), reverse=True)]

    cutoff_idx = n

    for i in range(cutoff_idx, len(lb_scores)):
        if lb_scores[i] < lb_scores[i-1]:
            break
        else:
            cutoff_idx += 1

    return lb_pep[:cutoff_idx]

In [62]:
def LBCyclopeptideSequencing(spectrum, n):
    
    parent_mass = spectrum[-1]
    leaderboard = [""]
    leader_pep = ""

    while len(leaderboard) != 0:
        leaderboard = Expand(leaderboard)

        to_remove = set()
        for pep in leaderboard:
            if Mass(pep, amino_acid_mass) == parent_mass:
                if CyclopeptideScore(pep, spectrum) > CyclopeptideScore(leader_pep, spectrum):
                    leader_pep = pep
            elif Mass(pep, amino_acid_mass) > parent_mass:
                to_remove.add(pep)

        for i in to_remove:
            leaderboard.remove(i)

        leaderboard = Trim(leaderboard, spectrum, n)

    return DisplayPepMass(leader_pep)

### Test Cases

In [68]:
# Create a function for test suite
def TestSuite(function, cases):
    print("*"*50)
    print("TEST SUITE\n")
    passed = 0
    for i, case in enumerate(cases):
        spectrum, n, answer = case
        result = function(spectrum, n)
        if sorted(result) == sorted(answer):
            print("- Test Case {} Passed. Expected: {}, Actual: {}"
                  .format(i+1, answer, result))
            passed += 1
        else:
            print("- Test Case {} Failed. Expected: {}, Actual: {}"
                  .format(i+1, answer, result))
    print("\n{} out of {} passed.".format(passed, len(cases)), end=" ")
    print("END OF TEST SUITE.")
    print("*"*50)

In [69]:
# Create test cases to pass into test suite
case1 = ([0, 71, 113, 129, 147, 200, 218, 260, 313, 331, 347, 389, 460], 10, "113-147-71-129")

cases = [case1]

TestSuite(LBCyclopeptideSequencing, cases)

**************************************************
TEST SUITE

- Test Case 1 Passed. Expected: 113-147-71-129, Actual: 113-147-71-129

1 out of 1 passed. END OF TEST SUITE.
**************************************************


### Stepik Coding Exercise

In [66]:
sp1 = [0, 71, 113, 129, 147, 200, 218, 260, 313, 331, 347, 389, 460]

print(LBCyclopeptideSequencing(sp1, 10))

113-147-71-129


In [64]:
sp2 = [0, 71, 71, 71, 87, 97, 97, 99, 101, 103, 113, 113, 114, 115, 128, 128, 129, 137, 147, 163, 163, 170, 184, 184, 186, 186, 190, 211, 215, 226, 226, 229, 231, 238, 241, 244, 246, 257, 257, 276, 277, 278, 299, 300, 312, 316, 317, 318, 318, 323, 328, 340, 343, 344, 347, 349, 356, 366, 370, 373, 374, 391, 401, 414, 414, 415, 419, 427, 427, 431, 437, 441, 446, 453, 462, 462, 462, 470, 472, 502, 503, 503, 511, 515, 529, 530, 533, 533, 540, 543, 547, 556, 559, 569, 574, 575, 584, 590, 600, 600, 604, 612, 616, 617, 630, 640, 640, 643, 646, 648, 660, 671, 683, 684, 687, 693, 703, 703, 719, 719, 719, 729, 730, 731, 737, 740, 741, 745, 747, 754, 774, 780, 784, 790, 797, 800, 806, 818, 826, 827, 832, 833, 838, 846, 846, 847, 850, 868, 869, 877, 884, 889, 893, 897, 903, 908, 913, 917, 930, 940, 947, 956, 960, 960, 961, 964, 965, 966, 983, 983, 985, 1002, 1009, 1010, 1011, 1021, 1031, 1031, 1036, 1053, 1054, 1058, 1059, 1062, 1063, 1074, 1076, 1084, 1092, 1103, 1113, 1122, 1124, 1130, 1133, 1134, 1145, 1146, 1146, 1149, 1150, 1155, 1156, 1171, 1173, 1174, 1187, 1191, 1193, 1200, 1212, 1221, 1233, 1240, 1242, 1246, 1259, 1260, 1262, 1277, 1278, 1283, 1284, 1287, 1287, 1288, 1299, 1300, 1303, 1309, 1311, 1320, 1330, 1341, 1349, 1357, 1359, 1370, 1371, 1374, 1375, 1379, 1380, 1397, 1402, 1402, 1412, 1422, 1423, 1424, 1431, 1448, 1450, 1450, 1467, 1468, 1469, 1472, 1473, 1473, 1477, 1486, 1493, 1503, 1516, 1520, 1525, 1530, 1536, 1540, 1544, 1549, 1556, 1564, 1565, 1583, 1586, 1587, 1587, 1595, 1600, 1601, 1606, 1607, 1615, 1627, 1633, 1636, 1643, 1649, 1653, 1659, 1679, 1686, 1688, 1692, 1693, 1696, 1702, 1703, 1704, 1714, 1714, 1714, 1730, 1730, 1740, 1746, 1749, 1750, 1762, 1773, 1785, 1787, 1790, 1793, 1793, 1803, 1816, 1817, 1821, 1829, 1833, 1833, 1843, 1849, 1858, 1859, 1864, 1877, 1886, 1890, 1893, 1900, 1900, 1903, 1904, 1918, 1922, 1930, 1930, 1931, 1961, 1963, 1971, 1971, 1971, 1980, 1987, 1992, 1996, 2002, 2006, 2006, 2014, 2018, 2019, 2019, 2032, 2042, 2059, 2060, 2063, 2067, 2077, 2084, 2086, 2089, 2090, 2093, 2105, 2110, 2115, 2115, 2116, 2117, 2121, 2133, 2134, 2155, 2156, 2157, 2176, 2176, 2187, 2189, 2192, 2195, 2202, 2204, 2207, 2207, 2218, 2222, 2243, 2247, 2247, 2249, 2249, 2263, 2270, 2270, 2286, 2296, 2304, 2305, 2305, 2318, 2319, 2320, 2320, 2330, 2332, 2334, 2336, 2336, 2346, 2362, 2362, 2362, 2433]

print(LBCyclopeptideSequencing(sp2, 325))

163-114-97-129-97-147-99-71-186-71-113-163-115-71-113-128-103-87-128-101-137


In [70]:
string = "0 71 71 71 97 103 113 113 115 115 128 128 128 129 129 129 131 142 156 163 174 184 186 186 186 226 227 228 232 234 243 244 246 255 257 259 276 283 284 298 299 299 303 314 314 315 315 329 337 347 355 356 370 374 375 400 411 411 412 413 426 427 427 429 430 443 444 445 450 462 466 484 503 515 526 533 539 540 542 542 542 555 559 560 561 563 572 579 586 590 599 613 636 643 654 654 655 657 661 668 670 671 676 684 687 689 689 690 705 714 726 728 741 749 764 765 774 776 782 783 786 786 797 799 818 818 818 840 843 845 854 862 862 870 875 877 879 889 889 889 893 911 914 915 946 953 960 960 960 968 969 972 974 983 985 990 990 992 1004 1008 1008 1018 1031 1045 1048 1063 1081 1089 1097 1097 1098 1098 1101 1101 1102 1105 1116 1116 1121 1121 1123 1132 1134 1173 1176 1176 1187 1192 1204 1210 1213 1218 1226 1229 1229 1230 1236 1244 1244 1252 1263 1275 1283 1288 1289 1290 1291 1304 1307 1315 1333 1341 1344 1357 1358 1359 1360 1365 1373 1385 1396 1404 1404 1412 1418 1419 1419 1422 1430 1435 1438 1444 1456 1461 1472 1472 1475 1514 1516 1525 1527 1527 1532 1532 1543 1546 1547 1547 1550 1550 1551 1551 1559 1567 1585 1600 1603 1617 1630 1640 1640 1644 1656 1658 1658 1663 1665 1674 1676 1679 1680 1688 1688 1688 1695 1702 1733 1734 1737 1755 1759 1759 1759 1769 1771 1773 1778 1786 1786 1794 1803 1805 1808 1830 1830 1830 1849 1851 1862 1862 1865 1866 1872 1874 1883 1884 1899 1907 1920 1922 1934 1943 1958 1959 1959 1961 1964 1972 1977 1978 1980 1987 1991 1993 1994 1994 2005 2012 2035 2049 2058 2062 2069 2076 2085 2087 2088 2089 2093 2106 2106 2106 2108 2109 2115 2122 2133 2145 2150 2164 2182 2186 2198 2203 2204 2205 2218 2219 2221 2221 2222 2235 2236 2237 2237 2248 2273 2274 2278 2292 2293 2301 2311 2319 2333 2333 2334 2334 2345 2349 2349 2350 2364 2365 2372 2389 2391 2393 2402 2404 2405 2414 2416 2420 2421 2422 2462 2462 2462 2464 2474 2485 2492 2506 2517 2519 2519 2519 2520 2520 2520 2533 2533 2535 2535 2545 2551 2577 2577 2577 2648"

ret = string.replace(" ", ", ")
print(ret)

0, 71, 71, 71, 97, 103, 113, 113, 115, 115, 128, 128, 128, 129, 129, 129, 131, 142, 156, 163, 174, 184, 186, 186, 186, 226, 227, 228, 232, 234, 243, 244, 246, 255, 257, 259, 276, 283, 284, 298, 299, 299, 303, 314, 314, 315, 315, 329, 337, 347, 355, 356, 370, 374, 375, 400, 411, 411, 412, 413, 426, 427, 427, 429, 430, 443, 444, 445, 450, 462, 466, 484, 503, 515, 526, 533, 539, 540, 542, 542, 542, 555, 559, 560, 561, 563, 572, 579, 586, 590, 599, 613, 636, 643, 654, 654, 655, 657, 661, 668, 670, 671, 676, 684, 687, 689, 689, 690, 705, 714, 726, 728, 741, 749, 764, 765, 774, 776, 782, 783, 786, 786, 797, 799, 818, 818, 818, 840, 843, 845, 854, 862, 862, 870, 875, 877, 879, 889, 889, 889, 893, 911, 914, 915, 946, 953, 960, 960, 960, 968, 969, 972, 974, 983, 985, 990, 990, 992, 1004, 1008, 1008, 1018, 1031, 1045, 1048, 1063, 1081, 1089, 1097, 1097, 1098, 1098, 1101, 1101, 1102, 1105, 1116, 1116, 1121, 1121, 1123, 1132, 1134, 1173, 1176, 1176, 1187, 1192, 1204, 1210, 1213, 1218, 1226, 1229, 

In [71]:
sp3 = [0, 71, 71, 71, 97, 103, 113, 113, 115, 115, 128, 128, 128, 129, 129, 129, 131, 142, 156, 163, 174, 184, 186, 186, 186, 226, 227, 228, 232, 234, 243, 244, 246, 255, 257, 259, 276, 283, 284, 298, 299, 299, 303, 314, 314, 315, 315, 329, 337, 347, 355, 356, 370, 374, 375, 400, 411, 411, 412, 413, 426, 427, 427, 429, 430, 443, 444, 445, 450, 462, 466, 484, 503, 515, 526, 533, 539, 540, 542, 542, 542, 555, 559, 560, 561, 563, 572, 579, 586, 590, 599, 613, 636, 643, 654, 654, 655, 657, 661, 668, 670, 671, 676, 684, 687, 689, 689, 690, 705, 714, 726, 728, 741, 749, 764, 765, 774, 776, 782, 783, 786, 786, 797, 799, 818, 818, 818, 840, 843, 845, 854, 862, 862, 870, 875, 877, 879, 889, 889, 889, 893, 911, 914, 915, 946, 953, 960, 960, 960, 968, 969, 972, 974, 983, 985, 990, 990, 992, 1004, 1008, 1008, 1018, 1031, 1045, 1048, 1063, 1081, 1089, 1097, 1097, 1098, 1098, 1101, 1101, 1102, 1105, 1116, 1116, 1121, 1121, 1123, 1132, 1134, 1173, 1176, 1176, 1187, 1192, 1204, 1210, 1213, 1218, 1226, 1229, 1229, 1230, 1236, 1244, 1244, 1252, 1263, 1275, 1283, 1288, 1289, 1290, 1291, 1304, 1307, 1315, 1333, 1341, 1344, 1357, 1358, 1359, 1360, 1365, 1373, 1385, 1396, 1404, 1404, 1412, 1418, 1419, 1419, 1422, 1430, 1435, 1438, 1444, 1456, 1461, 1472, 1472, 1475, 1514, 1516, 1525, 1527, 1527, 1532, 1532, 1543, 1546, 1547, 1547, 1550, 1550, 1551, 1551, 1559, 1567, 1585, 1600, 1603, 1617, 1630, 1640, 1640, 1644, 1656, 1658, 1658, 1663, 1665, 1674, 1676, 1679, 1680, 1688, 1688, 1688, 1695, 1702, 1733, 1734, 1737, 1755, 1759, 1759, 1759, 1769, 1771, 1773, 1778, 1786, 1786, 1794, 1803, 1805, 1808, 1830, 1830, 1830, 1849, 1851, 1862, 1862, 1865, 1866, 1872, 1874, 1883, 1884, 1899, 1907, 1920, 1922, 1934, 1943, 1958, 1959, 1959, 1961, 1964, 1972, 1977, 1978, 1980, 1987, 1991, 1993, 1994, 1994, 2005, 2012, 2035, 2049, 2058, 2062, 2069, 2076, 2085, 2087, 2088, 2089, 2093, 2106, 2106, 2106, 2108, 2109, 2115, 2122, 2133, 2145, 2150, 2164, 2182, 2186, 2198, 2203, 2204, 2205, 2218, 2219, 2221, 2221, 2222, 2235, 2236, 2237, 2237, 2248, 2273, 2274, 2278, 2292, 2293, 2301, 2311, 2319, 2333, 2333, 2334, 2334, 2345, 2349, 2349, 2350, 2364, 2365, 2372, 2389, 2391, 2393, 2402, 2404, 2405, 2414, 2416, 2420, 2421, 2422, 2462, 2462, 2462, 2464, 2474, 2485, 2492, 2506, 2517, 2519, 2519, 2519, 2520, 2520, 2520, 2533, 2533, 2535, 2535, 2545, 2551, 2577, 2577, 2577, 2648]

print(LBCyclopeptideSequencing(sp3, 243))

186-113-163-71-103-129-97-186-128-131-115-129-186-129-128-156-71-71-113-115-128
